In [1]:
import avlib.build.avlib as av
import numpy as np
from PIL import Image

In [2]:
generator = av.Generator("Fast.and.the.Furious.Tokyo.Drift.mp4", (1280, 720), 8)

[h264_cuvid @ 0x56448d081200] Invalid pkt_timebase, passing timestamps as-is.
[h264_cuvid @ 0x56448f336b00] Invalid pkt_timebase, passing timestamps as-is.


In [ ]:
for i in range(1):
  x, y = generator.generate_batch()
  if x is None:
    print("end of stream ", i)
    break

In [ ]:
Image.fromarray(y[0])

In [3]:
import keras
from keras.api.layers import Input, Rescaling, Conv2D, Conv2DTranspose, Concatenate, LeakyReLU, Activation, BatchNormalization
from keras.api.models import Model

2024-06-09 21:15:48.473158: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-09 21:15:48.494229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
m_in = Input(shape=(720, 1280, 3))
# m_resc0 = Rescaling(scale=1.0/255.0)(m_in)

# (720, 1280, 3)
m_enc0_c = Conv2D(64, kernel_size=3, strides=2, padding="same")(m_in)
m_enc0_a = LeakyReLU()(m_enc0_c)
# (360, 640, 64)

# (360, 640, 64)
m_enc1_c = Conv2D(128, kernel_size=3, strides=2, padding="same")(m_enc0_a)
m_enc1_a = LeakyReLU()(m_enc1_c)
# (180, 320, 64)

# (180, 320, 64)
m_enc2_c = Conv2D(256, kernel_size=3, strides=2, padding="same")(m_enc1_a)
m_enc2_a = LeakyReLU()(m_enc2_c)
# (90, 160, 64)

# (90, 160, 64)
m_enc3_c = Conv2D(512, kernel_size=3, strides=2, padding="same")(m_enc2_a)
m_enc3_a = LeakyReLU()(m_enc3_c)
# (45, 80, 512)

# (45, 80, 512)
m_dec0_c = Conv2DTranspose(256, kernel_size=3, strides=2, padding="same")(m_enc3_a)
m_dec0_a = LeakyReLU()(m_dec0_c)
# (90, 160, 64)

# (90, 160, 64)
m_dec1_i = Concatenate()([m_dec0_a, m_enc2_a])
m_dec1_c = Conv2DTranspose(128, kernel_size=3, strides=2, padding="same")(m_dec1_i)
m_dec1_a = LeakyReLU()(m_dec1_c)
# (180, 320, 64)

# (180, 320, 64)
m_dec2_i = Concatenate()([m_dec1_a, m_enc1_a])
m_dec2_c = Conv2DTranspose(64, kernel_size=3, strides=2, padding="same")(m_dec2_i)
m_dec2_a = LeakyReLU()(m_dec2_c)
# (360, 640, 64)

# (360, 640, 64)
m_dec3_i = Concatenate()([m_dec2_a, m_enc0_a])
m_dec3_c = Conv2DTranspose(3, kernel_size=3, strides=2, padding="same")(m_dec3_i)
m_dec3_a = Activation("sigmoid")(m_dec3_c)
# (720, 1280, 3)

# m_out = Rescaling(scale=255.0)(m_dec3_a)
model = Model(m_in, m_dec3_a)

In [5]:
model = keras.api.saving.load_model("model.keras")

ValueError: File not found: filepath=model.keras. Please ensure the file is an accessible `.keras` zip file.

In [7]:
model.compile("adam", "mse")

In [9]:
running = True
while running:
  for i in range(50):
    x, y = generator.generate_batch()
    if x is None:
      print("end of stream ", i)
      running = False
      break
    x = x[:, :, :, :3].astype(np.float32) / 256.0
    y = y[:, :, :, :3].astype(np.float32) / 256.0
    logs = model.train_on_batch(x, y)
    print(logs)
  model.save("model.keras", overwrite=True)

TypeError: Exception encountered when calling Conv2D.call().

[1mValue passed to parameter 'input' has DataType uint8 not in list of allowed values: float16, bfloat16, float32, float64, int32[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(8, 720, 1280, 3), dtype=uint8)

In [10]:
x

array([[[[0.27734375, 0.21484375, 0.140625  ],
         [0.25390625, 0.19140625, 0.1171875 ],
         [0.2421875 , 0.16796875, 0.1171875 ],
         ...,
         [0.        , 0.        , 0.00390625],
         [0.        , 0.        , 0.00390625],
         [0.        , 0.        , 0.00390625]],

        [[0.28125   , 0.21875   , 0.14453125],
         [0.26171875, 0.19921875, 0.125     ],
         [0.24609375, 0.171875  , 0.12109375],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.2890625 , 0.2265625 , 0.15234375],
         [0.265625  , 0.203125  , 0.12890625],
         [0.25      , 0.17578125, 0.125     ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.53125   , 0.36328125, 0.33984375],
         [0.52734375, 0.359375  , 0.3359375 ]

In [ ]:
x, y = generator.generate_batch()
x = x[:, :, :, :3]

In [ ]:
t = model.predict(x)

In [ ]:
Image.fromarray((t[7]).astype(np.uint8))